In [104]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Amog\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Amog\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [105]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [106]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [107]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

31 documents
6 classes ['about', 'ccs', 'goodbye', 'greeting', 'options', 'thanks']
55 unique lemmatized words ["'s", ',', 'anyone', 'apply', 'are', 'awesome', 'be', 'bye', 'can', 'cc', 'chatting', 'could', 'day', 'do', 'doe', 'expect', 'for', 'from', 'get', 'good', 'goodbye', 'happens', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'into', 'is', 'later', 'me', 'next', 'nice', 'offered', 'procedure', 'provide', 'see', 'siam', 'stand', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'what', 'you']


In [108]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [109]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
31/31 [==============================] - 0s 12ms/step - loss: 1.8833 - accuracy: 0.0968
Epoch 2/200
31/31 [==============================] - 0s 869us/step - loss: 1.8144 - accuracy: 0.1613
Epoch 3/200
31/31 [==============================] - 0s 2ms/step - loss: 1.7821 - accuracy: 0.2258
Epoch 4/200
31/31 [==============================] - 0s 1ms/step - loss: 1.6605 - accuracy: 0.2581
Epoch 5/200
31/31 [==============================] - 0s 2ms/step - loss: 1.6209 - accuracy: 0.4194
Epoch 6/200
31/31 [==============================] - 0s 2ms/step - loss: 1.5682 - accuracy: 0.4194
Epoch 7/200
31/31 [==============================] - 0s 2ms/step - loss: 1.5850 - accuracy: 0.5161
Epoch 8/200
31/31 [==============================] - 0s 2ms/step - loss: 1.5130 - accuracy: 0.4516
Epoch 9/200
31/31 [==============================] - 0s 3ms/step - loss: 1.4133 - accuracy: 0.4516
Epoch 10/200
31/31 [==============================] - 0s 3ms/step - loss: 1.4394 - accuracy: 0.3871
Epoch 

31/31 [==============================] - 0s 804us/step - loss: 0.0254 - accuracy: 1.0000
Epoch 161/200
31/31 [==============================] - 0s 836us/step - loss: 0.0030 - accuracy: 1.0000
Epoch 162/200
31/31 [==============================] - 0s 611us/step - loss: 0.0592 - accuracy: 0.9677
Epoch 163/200
31/31 [==============================] - 0s 579us/step - loss: 0.0241 - accuracy: 1.0000
Epoch 164/200
31/31 [==============================] - 0s 579us/step - loss: 0.0083 - accuracy: 1.0000
Epoch 165/200
31/31 [==============================] - 0s 900us/step - loss: 0.0080 - accuracy: 1.0000
Epoch 166/200
31/31 [==============================] - 0s 1ms/step - loss: 0.0241 - accuracy: 1.0000
Epoch 167/200
31/31 [==============================] - 0s 1ms/step - loss: 0.0344 - accuracy: 1.0000
Epoch 168/200
31/31 [==============================] - 0s 1ms/step - loss: 0.0145 - accuracy: 1.0000
Epoch 169/200
31/31 [==============================] - 0s 869us/step - loss: 0.0125 - accurac

In [110]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [111]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("SIAM CHATBOT")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()